# 导入模型

OSError: Incorrect path_or_model_id: '../model/embedding-model/acge_text_embedding/'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [28]:

from sentence_transformers import SentenceTransformer

# 指定本地模型路径
local_model_path = "../model/embedding-model/models--aspire--acge_text_embedding/snapshots/16582d03bb81daeae334c3a73d388d270c46a32a"

# 加载本地模型
embeding_model = SentenceTransformer(local_model_path)

# 加载数据

In [22]:
import  json
import pdfplumber

In [18]:
questions = json.load(open('./大数据导论知识问答/question_list.json', 'r', encoding='utf-8'))
pdf=pdfplumber.open('./大数据导论知识问答/大数据导论.pdf')
pdf_content = []
for page_idx in range(len(pdf.pages)):
    pdf_content.append({
        'page': 'page_'+ str(page_idx+1),
        'content': pdf.pages[page_idx].extract_text()
    })

In [29]:
#建立数组
questions_sentences = [x['question'] for x in questions]
pdf_sentences = [x['content'] for x in pdf_content]
#embeding 嵌入数据
questions_embeddings = embeding_model.encode(questions_sentences, normalize_embeddings=True)  #normalize_embeddings=True 归一化
pdf_embeddings = embeding_model.encode(pdf_sentences, normalize_embeddings=True)  #同上
# 开始语义检索
for query_idx, feat in enumerate(questions_embeddings):
    # 矩阵运算
    scores = feat @ pdf_embeddings.T
    max_socre_idx = scores.argmax()+1
    questions[query_idx]['reference'] = 'page_'+str(max_socre_idx)
#保存数据
json.dump(questions, open('anwser_NLP.json', 'w', encoding='utf-8'), ensure_ascii=False)

# KIMI AI